- version 45: groupkfold with truncation by oof

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import seaborn as sns
import scipy as sp
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
        
    def fit(self, X, y,random_flg=False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(0.5,0.6), np.random.uniform(0.6,0.7), np.random.uniform(0.8,0.9)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [4]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 8s, sys: 9.91 s, total: 1min 18s
Wall time: 1min 18s


In [5]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]

In [6]:
assess_title = ['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)']
def remove_index_calc(df):
    additional_remove_index = []
    for i, session in df.groupby('installation_id', sort=False):
        last_row = session.index[-1]
        session = session[session.title.isin(assess_title)]
        first_row = session.index[-1] + 1
        for j in range(first_row, last_row+1):
            additional_remove_index.append(j)                
    return additional_remove_index
additional_remove_index = remove_index_calc(train)
train = train[~train.index.isin(additional_remove_index)]

# Preprocess and Feature engineering

In [7]:
#length_list = [19, 17,  61 ,-999,61, -999, 109, 25, -999, 80, -999, -999, 156, 26, -999, 126, -999, -999, 20, -999, 60, -999, 22, -999,
#              -999, -999, -999, -999, 18, -999, 72, -999, 118, 24, 142, -999, -999, -999, 19, 61] 
#title_list = ["Welcome to Lost Lagoon!",  "Tree Top City - Level 1", "Ordering Spheres", "All Star Sorting",  "Costume Box", "Fireworks (Activity)", 
#              "12 Monkeys", "Tree Top City - Level 2", "Flower Waterer (Activity)", "Pirate's Tale",  "Mushroom Sorter (Assessment)",  "Air Show",  
#              "Treasure Map", "Tree Top City - Level 3", "Crystals Rule", "Rulers", "Bug Measurer (Activity)",  "Bird Measurer (Assessment)",  
#              "Magma Peak - Level 1",   "Sandcastle Builder (Activity)",  "Slop Problem",  "Scrub-A-Dub",  "Watering Hole (Activity)", 
#              "Magma Peak - Level 2" , "Dino Drink", "Bubble Bath",  "Bottle Filler (Activity)",  "Dino Dive", "Cauldron Filler (Assessment)", 
#              "Crystal Caves - Level 1", "Chow Time", "Balancing Act",  "Chicken Balancer (Activity)",  "Lifting Heavy Things", 
#              "Crystal Caves - Level 2" , "Honey Cake" ,  "Happy Camel", "Cart Balancer (Assessment)",  "Leaf Leader",  "Crystal Caves - Level 3",  
#              "Heavy, Heavier, Heaviest"]
#title_list = pd.DataFrame(title_list)
#length_list = pd.DataFrame(length_list)
#length_df = pd.concat([title_list, length_list], axis=1)
#length_df.columns = ["title", "length"]

In [8]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique())))
    
    list_of_eventid = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))

    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    
    #train["x"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else np.nan)
    #train["y"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else np.nan)
    #test["x"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else np.nan)
    #test["y"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else np.nan)
                
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_eventid

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_eventid = encode_title(train, test)

CPU times: user 37.9 s, sys: 3.02 s, total: 41 s
Wall time: 40.8 s


In [9]:
# activities_world = {'CRYSTALCAVES': 0, 'NONE': 1, 'TREETOPCITY': 2, 'MAGMAPEAK': 3}
def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    eventid_count = {str(ele): 0 for ele in list_of_eventid}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    
    last_session_time_sec = 0
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    miss = 0
    crys_game_true = 0; crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    crys_act_true = 0; crys_act_false = 0
    tree_act_true = 0; tree_act_false = 0
    magma_act_true = 0; magma_act_false = 0
    crys_act_acc = []; tree_act_acc = []; magma_act_acc = []
    #x = np.array([]); y = np.array([])
        
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent   
            
            if session_type == "Game":
                true = session['event_data'].str.contains('true').sum()
                false = session['event_data'].str.contains('false').sum() 
                if session_world == 0:
                    crys_game_true += true
                    crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                elif session_world == 2:
                    tree_game_true += true
                    tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                elif session_world == 3:
                    magma_game_true += true
                    magma_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                else:
                    pass
                
            if session_type == "Activity":
                true = session['event_data'].str.contains('true').sum()
                false = session['event_data'].str.contains('false').sum() 
                if session_world == 0:
                    crys_act_true += true
                    crys_act_false += false
                    crys_act_acc.append(true / (true + false) if (true + false) != 0 else 0)
                elif session_world == 2:
                    tree_act_true += true
                    tree_act_false += false
                    tree_act_acc.append(true / (true + false) if (true + false) != 0 else 0)
                elif session_world == 3:
                    magma_act_true += true
                    magma_act_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                else:
                    pass

        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features.update(time_spent_each_act.copy()) # cumulative gameplay time in each title
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features.update(eventid_count.copy()) # apperance of eventid
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            if session_world == 0:
                features["game_true"] = crys_game_true
                features["game_false"] = crys_game_false
                features['game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
                features["act_true"] = crys_act_true
                features["act_false"] = crys_act_false
                features['act_accuracy'] = crys_act_true / (crys_act_true + crys_act_false) if (crys_act_true + crys_act_false) != 0 else 0
                features["act_accuracy_std"] = np.std(crys_act_acc) if len(crys_act_acc) >=1 else 0
                features["last_act_acc"] = crys_act_acc[-1] if len(crys_act_acc) >=1 else 0
            elif session_world == 2:
                features["game_true"] = tree_game_true
                features["game_false"] = tree_game_false
                features['game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
                features["act_true"] = tree_act_true
                features["act_false"] = tree_act_false
                features['act_accuracy'] = tree_act_true / (tree_act_true + tree_act_false) if (tree_act_true + tree_act_false) != 0 else 0
                features["act_accuracy_std"] = np.std(tree_act_acc) if len(tree_act_acc) >=1 else 0
                features["last_act_acc"] = tree_act_acc[-1] if len(tree_act_acc) >=1 else 0
            elif session_world == 3:
                features["game_true"] = magma_game_true
                features["game_false"] = magma_game_false
                features['game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
                features["act_true"] = magma_act_true
                features["act_false"] = magma_act_false
                features['act_accuracy'] = magma_act_true / (magma_act_true + magma_act_false) if (magma_act_true + magma_act_false) != 0 else 0
                features["act_accuracy_std"] = np.std(magma_act_acc) if len(magma_act_acc) >=1 else 0
                features["last_act_acc"] = magma_act_acc[-1] if len(magma_act_acc) >=1 else 0
            
            #if len(x[~np.isnan(x)]) >=2:
            #    features["xrange_bet_assess"] = np.nanmax(x) - np.nanmin(x)
            #    features["yrange_bet_assess"] = np.nanmax(y) - np.nanmin(y)
            #else:
            #    features["xrange_bet_assess"] = 0
            #    features["yrange_bet_assess"] = 0
            #features["touch_range"] = features["xrange_bet_assess"] * features["yrange_bet_assess"] 
            # from the end of previous assessment to the start of next assessment --------------------------
                        
            # unique type --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            
            # nums in target assessment data ------------------------------------------
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds) 
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        miss += np.sum(session["misses"])
        n_of_eventid = Counter(session['event_id']) 
        for key in n_of_eventid.keys():
            eventid_count[str(key)] += n_of_eventid[key]
            
        user_world_count["world_"+str(session_world)] += session.shape[0]
        
        #x = np.concatenate([x, session["x"]], axis=0)
        #y = np.concatenate([y, session["y"]], axis=0)

        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [10]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)


(17690, 850)


In [11]:
del train

In [12]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)


(1000, 850)


In [13]:
del test

# Feature selection and modelling

In [14]:
#correlations = new_train.corr().abs()
#correlations = correlations.mask(np.tril(np.ones(correlations.shape)).astype(np.bool))
#correlations = correlations.stack().reset_index()
#corr_columns = ["level_0", "level_1", "value"]
#correlations.columns = corr_columns
#correlations = correlations.sort_values("value", ascending=False).reset_index(drop=True)

#high_corr = correlations[correlations["value"] == 1]

#high_corr_features = []
#for i in range(high_corr.shape[0]):
#    if high_corr.iloc[i]["level_0"] not in high_corr_features and high_corr.iloc[i]["level_1"] not in high_corr_features:
#        high_corr_features.append(high_corr.iloc[i]["level_0"])
#    elif high_corr.iloc[i]["level_0"] in high_corr_features and high_corr.iloc[i]["level_1"] not in high_corr_features:
#        high_corr_features.append(high_corr.iloc[i]["level_1"])
#    elif high_corr.iloc[i]["level_0"] not in high_corr_features and high_corr.iloc[i]["level_1"] in high_corr_features:
#        high_corr_features.append(high_corr.iloc[i]["level_0"])

In [15]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = []
for i in X_train.columns:
    if X_train[i].std() == 0 and i not in remove_features:
        remove_features.append(i)
X_train = X_train.drop(remove_features, axis=1)
X_train = X_train[sorted(X_train.columns.tolist())]
y_train = new_train.accuracy_group
print(X_train.shape)

(17690, 824)


In [16]:
n_folds=5
groups = np.array(X_train.installation_id)
gkf=GroupKFold(n_splits = n_folds)
coefficients = []
models = []
train_qwk_scores = []
test_qwk_scores = []
lgbm_params = {
    "objective" : "regression",
    "metric" : "rmse",
    "tree_learner": "serial",
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}

valid = np.array([])
real = np.array([])
features_list = [i for i in X_train.columns if i != "installation_id"]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
for i , (train_index, test_index) in enumerate(gkf.split(X_train, y_train, groups)):
    optR = OptimizedRounder()
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_train2 = X_train2.drop(['installation_id'],axis=1)
    
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    test2 = pd.concat([X_test2, y_test2], axis=1)
    test2 = test2.groupby('installation_id').apply(lambda x: x.sample(1, random_state=1223)).reset_index(drop=True)
    X_test2 = test2.drop(["accuracy_group", "installation_id"], axis=1)
    y_test2 = test2["accuracy_group"]
    
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        num_boost_round=100000,
        early_stopping_rounds=10,
    )
    
    models.append(clf)
    valid = np.concatenate((valid, np.array(clf.predict(X_test2, num_iteration = clf.best_iteration))), axis=0) 
    real = np.concatenate((real, np.array(y_test2)), axis=0) 
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

best_score = 0
for i in tqdm(range(50)):
    optR = OptimizedRounder()    
    optR.fit(np.array(valid).reshape(-1, ), np.array(real).reshape(-1, ), random_flg=True)
    tmp_coefficients = optR.coefficients()
    opt_preds = optR.predict(np.array(valid).reshape(-1, ), tmp_coefficients)
    qwk_score = qwk(real, opt_preds)
    if qwk_score > best_score:
        best_score = qwk_score
        coefficients = tmp_coefficients
print(best_score)
print(coefficients)

[1]	training's rmse: 1.25209	valid_1's rmse: 1.27617
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 1.2484	valid_1's rmse: 1.27299
[3]	training's rmse: 1.24478	valid_1's rmse: 1.26982
[4]	training's rmse: 1.2412	valid_1's rmse: 1.26668
[5]	training's rmse: 1.23769	valid_1's rmse: 1.26357
[6]	training's rmse: 1.23423	valid_1's rmse: 1.26051
[7]	training's rmse: 1.23087	valid_1's rmse: 1.25749
[8]	training's rmse: 1.22754	valid_1's rmse: 1.25463
[9]	training's rmse: 1.22424	valid_1's rmse: 1.25163
[10]	training's rmse: 1.22106	valid_1's rmse: 1.24891
[11]	training's rmse: 1.21788	valid_1's rmse: 1.2462
[12]	training's rmse: 1.21469	valid_1's rmse: 1.24343
[13]	training's rmse: 1.21161	valid_1's rmse: 1.2408
[14]	training's rmse: 1.20847	valid_1's rmse: 1.23803
[15]	training's rmse: 1.20549	valid_1's rmse: 1.2355
[16]	training's rmse: 1.20254	valid_1's rmse: 1.23298
[17]	training's rmse: 1.19959	valid_1's rmse: 1.23048
[18]	training's rmse: 1.19666	valid


0.55230802
[1.01921299 1.57702527 2.1905866 ]


In [17]:
feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True).head(500)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,session_title,527,687,538,529,575,571.2,60.433104,0.105800
1,accumulated_accuracy_group,437,640,412,554,628,534.2,94.620082,0.177125
2,27253bdc,258,386,349,289,451,346.6,68.738926,0.198324
3,game_accuracy,218,321,253,242,199,246.6,41.668213,0.168971
4,misses,179,227,214,213,313,229.2,44.812498,0.195517
5,7372e1a5,198,193,164,193,214,192.4,16.156732,0.083975
6,3afb49e6,136,187,183,179,189,174.8,19.701777,0.112710
7,duration_mean,178,294,124,84,142,164.4,71.525100,0.435068
8,act_true,136,193,131,139,138,147.4,22.966062,0.155808
9,game_true,134,173,156,105,141,141.8,22.763128,0.160530


In [18]:
print(X_train.shape)
print("")
print(", ".join('%s'%x for x in X_train.columns))

(17690, 824)

0, 0086365d, 00c73085, 01ca3a3c, 022b4259, 02a42007, 0330ab6a, 0413e89d, 04df9b66, 05ad839b, 06372577, 070a5291, 08fd73f3, 08ff79ad, 0a08139c, 0ce40006, 0d18d96c, 0d1da71f, 0db6d71d, 1, 119b5b02, 12 Monkeys_2000, 1325467d, 1340b8d7, 1375ccb7, 13f56524, 14de4c5d, 155f62a4, 1575e76c, 15a43e5b, 15ba1109, 15eb4a7d, 15f99afc, 160654fd, 16667cc5, 16dffff1, 17113b36, 19967db1, 1996c610, 1af8be29, 1b54d27f, 1bb5fbdb, 1beb320a, 1c178d24, 1cc7cfca, 1cf54632, 1f19558b, 2, 222660ff, 2230fab4, 250513af, 25fa8af4, 262136f4, 26a5a3dd, 26fd2d99, 27253bdc, 28520915, 28a4eb9a, 28ed704e, 28f975ea, 29a42aea, 29bdd9ba, 29f54413, 2a444e03, 2a512369, 2b058fe3, 2b9272f4, 2c4e6db0, 2dc29e21, 2dcad279, 2ec694de, 2fb91ec1, 3, 30614231, 30df3273, 31973d56, 3323d7e9, 33505eae, 3393b68b, 363c86c9, 363d3849, 36fa3ebe, 37937459, 37c53127, 37db1c2f, 37ee8496, 38074c54, 392e14df, 3a4be871, 3afb49e6, 3afde5dd, 3b2048ee, 3babcb9b, 3bb91ced, 3bb91dda, 3bf1cf26, 3bfd1a65, 3ccd3f02, 3d0b9317, 3d63345e, 3d8c61b

In [19]:
#import eli5
#from eli5.sklearn import PermutationImportance
#my_model = lgb.LGBMRegressor(**lgbm_params).fit(X_train2, y_train2)
#perm = PermutationImportance(my_model, random_state=1).fit(X_test2, y_test2)
#eli5.show_weights(perm, feature_names = X_test2.columns.tolist(), top = 500)

# prediction

In [20]:
X_test = new_test.drop(["installation_id", "accuracy_group"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
X_test = X_test[sorted(X_test.columns.tolist())]
pred_value = np.zeros([X_test.shape[0]])
for model in models:
    pred_value += model.predict(X_test, num_iteration = model.best_iteration) /n_folds
test_pred_class= optR.predict(pred_value.reshape(-1, ), coefficients)
sample_submission["accuracy_group"] = test_pred_class
sample_submission.to_csv('submission.csv', index=False)

In [21]:
sample_submission["accuracy_group"].value_counts(normalize = True)

3    0.471
2    0.281
0    0.145
1    0.103
Name: accuracy_group, dtype: float64